In [1]:
# loading the dslabs package and the murders dataset
library(dslabs)
data(murders)

# determining that the murders dataset is of the "data frame" class
class(murders)
# finding out more about the structure of the object
str(murders)
# showing the first 6 lines of the dataset
head(murders)

# using the accessor operator to obtain the population column
murders$population
# displaying the variable names in the murders dataset
names(murders)
# determining how many entries are in a vector
pop <- murders$population
length(pop)
# vectors can be of class numeric and character
class(pop)
class(murders$state)

# logical vectors are either TRUE or FALSE
z <- 3 == 2
z
class(z)

# factors are another type of class
class(murders$region)
# obtaining the levels of a factor
levels(murders$region)

[1] "data.frame"

'data.frame':	51 obs. of  5 variables:
 $ state     : chr  "Alabama" "Alaska" "Arizona" "Arkansas" ...
 $ abb       : chr  "AL" "AK" "AZ" "AR" ...
 $ region    : Factor w/ 4 levels "Northeast","South",..: 2 4 4 2 4 4 1 2 2 2 ...
 $ population: num  4779736 710231 6392017 2915918 37253956 ...
 $ total     : num  135 19 232 93 1257 ...


state,abb,region,population,total
Alabama,AL,South,4779736,135
Alaska,AK,West,710231,19
Arizona,AZ,West,6392017,232
Arkansas,AR,South,2915918,93
California,CA,West,37253956,1257
Colorado,CO,West,5029196,65


[1]  4779736   710231  6392017  2915918 37253956  5029196  3574097   897934
 [9]   601723 19687653  9920000  1360301  1567582 12830632  6483802  3046355
[17]  2853118  4339367  4533372  1328361  5773552  6547629  9883640  5303925
[25]  2967297  5988927   989415  1826341  2700551  1316470  8791894  2059179
[33] 19378102  9535483   672591 11536504  3751351  3831074 12702379  1052567
[41]  4625364   814180  6346105 25145561  2763885   625741  8001024  6724540
[49]  1852994  5686986   563626

[1] "state"      "abb"        "region"     "population" "total"

...

[1] "character"

[1] FALSE

[1] "logical"

[1] "factor"

[1] "Northeast"     "South"         "North Central" "West"

In [4]:
if("dplyr" %in% rownames(installed.packages()) == FALSE) {install.packages("dplyr")};library(dplyr)
if("tidyr" %in% rownames(installed.packages()) == FALSE) {install.packages("tidyr")};library(tidyr)

# If the dataset is not present in the current working directory then download it
fileUrl <- "https://d396qusza40orc.cloudfront.net/getdata%2Fprojectfiles%2FUCI%20HAR%20Dataset.zip"
if (!file.exists("UCI HAR Dataset")) {
    if (!file.exists("data")) {
        dir.create("data")
    }
    download.file(fileUrl, destfile="data/har.zip", method="curl")
    unzip("data/har.zip", exdir="./")
}

# ------------------------------------------------------------------------------
# Step 1 - Merges the training and the test sets to create one data set.
# ------------------------------------------------------------------------------

# training data
train_x <- read.table("UCI HAR Dataset//train/X_train.txt", nrows=7352, comment.char="")
train_sub <- read.table("UCI HAR Dataset//train/subject_train.txt", col.names=c("subject"))
train_y <- read.table("UCI HAR Dataset/train//y_train.txt", col.names=c("activity"))
train_data <- cbind(train_x, train_sub, train_y)

# test data
test_x <- read.table("UCI HAR Dataset//test/X_test.txt", nrows=2947, comment.char="")
test_sub <- read.table("UCI HAR Dataset/test/subject_test.txt", col.names=c("subject"))
test_y <- read.table("UCI HAR Dataset/test//y_test.txt", col.names=c("activity"))
test_data <- cbind(test_x, test_sub, test_y)

# merge both train and test data
data <- rbind(train_data, test_data)

# ------------------------------------------------------------------------------
# Step 2 - Extracts only the measurements on the mean and standard deviation for each measurement.
# ------------------------------------------------------------------------------
# read features
feature_list <- read.table("UCI HAR Dataset//features.txt", col.names = c("id", "name"))
features <- c(as.vector(feature_list[, "name"]), "subject", "activity")

# filter only features that has mean or std in the name
filtered_feature_ids <- grepl("mean|std|subject|activity", features) & !grepl("meanFreq", features)
filtered_data = data[, filtered_feature_ids]

# ------------------------------------------------------------------------------
# Step 3 - Uses descriptive activity names to name the activities in the data set
# ------------------------------------------------------------------------------
activities <- read.table("UCI HAR Dataset//activity_labels.txt", col.names=c("id", "name"))
for (i in 1:nrow(activities)) {
    filtered_data$activity[filtered_data$activity == activities[i, "id"]] <- as.character(activities[i, "name"])
}

# ------------------------------------------------------------------------------
# step 4 - Appropriately labels the data set with descriptive variable names.
# ------------------------------------------------------------------------------
# make feature names more human readble
filtered_feature_names <- features[filtered_feature_ids]
filtered_feature_names <- gsub("\\(\\)", "", filtered_feature_names)
filtered_feature_names <- gsub("Acc", "-acceleration", filtered_feature_names)
filtered_feature_names <- gsub("Mag", "-Magnitude", filtered_feature_names)
filtered_feature_names <- gsub("^t(.*)$", "\\1-time", filtered_feature_names)
filtered_feature_names <- gsub("^f(.*)$", "\\1-frequency", filtered_feature_names)
filtered_feature_names <- gsub("(Jerk|Gyro)", "-\\1", filtered_feature_names)
filtered_feature_names <- gsub("BodyBody", "Body", filtered_feature_names)
filtered_feature_names <- tolower(filtered_feature_names)

# assign names to features
names(filtered_data) <- filtered_feature_names

# ------------------------------------------------------------------------------
# step 5 - From the data set in step 4, creates a second, independent tidy data set with the average of each variable for each activity and each subject.
# ------------------------------------------------------------------------------
tidy_data <- tbl_df(filtered_data) %>%
    group_by('subject', 'activity') %>%
    summarise_each(funs(mean)) %>%
    gather(measurement, mean, -activity, -subject)

# Save the data into the file
write.table(tidy_data, file="tidy_data.txt", row.name=FALSE)

Warning message in mean.default(activity):
“argument is not numeric or logical: returning NA”

In [5]:
str(tidy_data)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	68 obs. of  4 variables:
 $ subject    : num  16.1 16.1 16.1 16.1 16.1 ...
 $ activity   : num  NA NA NA NA NA NA NA NA NA NA ...
 $ measurement: chr  "\"subject\"" "\"activity\"" "body-acceleration-mean-x-time" "body-acceleration-mean-y-time" ...
 $ mean       : chr  "subject" "activity" "0.274347260646063" "-0.0177434918458972" ...


In [6]:
names(tidy_data)

[1] "subject"     "activity"    "measurement" "mean"

In [7]:
length(tidy_data$measurement)

[1] 68

In [8]:
tidy_data$measurement

[1] "\"subject\""                                    
 [2] "\"activity\""                                   
 [3] "body-acceleration-mean-x-time"                  
 [4] "body-acceleration-mean-y-time"                  
 [5] "body-acceleration-mean-z-time"                  
 [6] "body-acceleration-std-x-time"                   
 [7] "body-acceleration-std-y-time"                   
 [8] "body-acceleration-std-z-time"                   
 [9] "gravity-acceleration-mean-x-time"               
[10] "gravity-acceleration-mean-y-time"               
[11] "gravity-acceleration-mean-z-time"               
[12] "gravity-acceleration-std-x-time"                
[13] "gravity-acceleration-std-y-time"                
[14] "gravity-acceleration-std-z-time"                
[15] "body-acceleration-jerk-mean-x-time"             
[16] "body-acceleration-jerk-mean-y-time"             
[17] "body-acceleration-jerk-mean-z-time"             
[18] "body-acceleration-jerk-std-x-time"              
[19] "body-acceleration-jerk-std-y-time"              
[20] "body-acceleration-jerk-std-z-time"              
[21] "body-gyro-mean-x-time"                          
[22] "body-gyro-mean-y-time"                          
[23] "body-gyro-mean-z-time"                          
[24] "body-gyro-std-x-time"                           
[25] "body-gyro-std-y-time"                           
[26] "body-gyro-std-z-time"                           
[27] "body-gyro-jerk-mean-x-time"                     
[28] "body-gyro-jerk-mean-y-time"                     
[29] "body-gyro-jerk-mean-z-time"                     
[30] "body-gyro-jerk-std-x-time"                      
[31] "body-gyro-jerk-std-y-time"                      
[32] "body-gyro-jerk-std-z-time"                      
[33] "body-acceleration-magnitude-mean-time"          
[34] "body-acceleration-magnitude-std-time"           
[35] "gravity-acceleration-magnitude-mean-time"       
[36] "gravity-acceleration-magnitude-std-time"        
[37] "body-acceleration-jerk-magnitude-mean-time"     
[38] "body-acceleration-jerk-magnitude-std-time"      
[39] "body-gyro-magnitude-mean-time"                  
[40] "body-gyro-magnitude-std-time"                   
[41] "body-gyro-jerk-magnitude-mean-time"             
[42] "body-gyro-jerk-magnitude-std-time"              
[43] "body-acceleration-mean-x-frequency"             
[44] "body-acceleration-mean-y-frequency"             
[45] "body-acceleration-mean-z-frequency"             
[46] "body-acceleration-std-x-frequency"              
[47] "body-acceleration-std-y-frequency"              
[48] "body-acceleration-std-z-frequency"              
[49] "body-acceleration-jerk-mean-x-frequency"        
[50] "body-acceleration-jerk-mean-y-frequency"        
[51] "body-acceleration-jerk-mean-z-frequency"        
[52] "body-acceleration-jerk-std-x-frequency"         
[53] "body-acceleration-jerk-std-y-frequency"         
[54] "body-acceleration-jerk-std-z-frequency"         
[55] "body-gyro-mean-x-frequency"                     
[56] "body-gyro-mean-y-frequency"                     
[57] "body-gyro-mean-z-frequency"                     
[58] "body-gyro-std-x-frequency"                      
[59] "body-gyro-std-y-frequency"                      
[60] "body-gyro-std-z-frequency"                      
[61] "body-acceleration-magnitude-mean-frequency"     
[62] "body-acceleration-magnitude-std-frequency"      
[63] "body-acceleration-jerk-magnitude-mean-frequency"
[64] "body-acceleration-jerk-magnitude-std-frequency" 
[65] "body-gyro-magnitude-mean-frequency"             
[66] "body-gyro-magnitude-std-frequency"              
[67] "body-gyro-jerk-magnitude-mean-frequency"        
[68] "body-gyro-jerk-magnitude-std-frequency"